In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import pickle
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
matplotlib 2.1.2
numpy 1.19.1
pandas 0.22.0
sklearn 0.19.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [2]:
# tf.function and auto-graph
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))
# 普通函数实现
print(scaled_elu(tf.constant(-3.0)))
print(scaled_elu(tf.constant([-3., -2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [3]:
# 基于graph的实现
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.0)))
print(scaled_elu_tf(tf.constant([-3., -2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [6]:
%timeit scaled_elu(tf.random.normal((1000,1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

300 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
422 µs ± 7.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
# 1 + 1/2 + 1/2^2 + ... + 1/2^n

@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [8]:
# 查看将一般的python代码转化为tensorflow的graph之后的代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [9]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        is_positive = ag__.converted_call(tf.greater_equal, (z, 0.0), None, fscope)
        try:
            do_return = True
            retval_ = fscope.mark_return_value((scale * ag__.converted_call(tf.where, (is_positive, z, (alpha * ag__.converted_call(tf.nn.elu, (z,), None, fscope))), None, fscope)))
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)

```

In [10]:
var = tf.Variable(0.)

# 将普通的python函数或者代码块转换为tensorflow中库函数或者graph
@tf.function
def add_21():
    return var.assign_add(21) # +=

print(add_21())   

tf.Tensor(21.0, shape=(), dtype=float32)


In [11]:
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z,3)

try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


从上面的运行结果可以看出，参数类型不满足的时候会报错

### 生成具体函数

具体函数定义的图表，可以被转化成 TensorFlow Lite 的模型或者导出到 SavedModel。为了从多态函数中导出具体函数，首先需要定义签名。签名可以根据以下方式定义：

- 在 tf.function 中定义 input_signature 参数。
- 将 tf.TensorSpec 传递给 get_concrete_function：例如 tf.TensorSpec（shape = [1]，dtype = tf.float32）。
- 将样本输入量传递给 get_concrete_function：例如 tf.constant（1。，shape = [1]）。

下面的例子展示了如何在 tf.function 中input_signature 参数。

官方链接：https://www.tensorflow.org/lite/convert/concrete_function?hl=zh-cn

In [14]:
class BasicModel(tf.Module):

    def __init__(self):
        self.const = None

    @tf.function(input_signature=[tf.TensorSpec(shape=[1], dtype=tf.float32)])
    def pow(self, x):
        if self.const is None:
            self.const = tf.Variable(2.)
        return x ** self.const

# 创建 tf.Module 对象。
root = BasicModel()

# 获取具体函数。
concrete_func = root.pow.get_concrete_function()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


下面的示例展示了将样本输入量传递给 get_concrete_function。

In [15]:
# 创建 tf.Module 对象。
root = tf.Module()
root.const = tf.Variable(2.)
root.pow = tf.function(lambda x : x ** root.const)

# 获取具体函数。
input_data = tf.constant(1., shape=[1])
concrete_func = root.pow.get_concrete_function(input_data)

In [16]:
# tf.function py func -> tf graph
# get_concrate_function -> add input signature -> SavedModel
cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32)
)
print(cube_func_int32)

In [18]:
print(cube_func_int32 is cube.get_concrete_function(
    tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(
    tf.constant([1,2,3])))


True
True


In [19]:
cube_func_int32.graph

In [20]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [22]:
# 获取具体的某一个op
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [23]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [24]:
cube_func_int32.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [25]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [26]:
# 获取graph中的所有node信息
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 175
}